In [ ]:
from numba import cuda
import numpy as np
import math

# heat sources used to create the border values
heat_sources = [ { 'x': 0.0, 'y': 0.0, 'range': 1.0, 'temp': 2.5 }, {'x': 0.5, 'y': 1.0, 'range': 1.0, 'temp': 2.5} ] 


# Apply the heat sources defined above. The handling of each border could be improved, but it works...
# Note that the order of the sources is important, values are overwritten, not modified
# There is no need to modify this function unless you find a bug
def apply_hs(m, hs, border):
    if border == 'top' or border == 'bottom':
        r = (0, m.shape[0])
    elif border == 'left' or border == 'right':
        r = (1, m.shape[0]-1)
    else:
        return

    for i in range(*r):
        if border == 'top':
            dist = math.sqrt(pow(i / (m.shape[0] - 1) - hs['x'], 2) + pow(hs['y'], 2))
        elif border == 'bottom':
            dist = math.sqrt(pow(i / (m.shape[0] - 1) - hs['x'], 2) + pow(1 - hs['y'], 2))
        elif border == 'left':
            dist = math.sqrt(pow(hs['x'], 2) + pow(i / (m.shape[0] - 1) - hs['y'], 2))
        elif border == 'right':
            dist = math.sqrt(pow(1 - hs['x'], 2) + pow(i / (m.shape[0] - 1) - hs['y'], 2))

        if dist <= hs['range']:
            if border == 'top':
                m[0, i] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'bottom':
                m[-1, i] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'left':
                m[i, 0] += (hs['range'] - dist) / hs['range'] * hs['temp']
            elif border == 'right':
                m[i,-1] += (hs['range'] - dist) / hs['range'] * hs['temp']


# Create a grid and apply the heat sources
def init(hs, n):
    num_p = n + 2

    m = np.zeros((num_p, num_p))
    for hs in heat_sources:
        apply_hs(m, hs, 'top')
        apply_hs(m, hs, 'bottom')
        apply_hs(m, hs, 'left')
        apply_hs(m, hs, 'right')

    return m


@cuda.jit
def update_cells_by_color_chessboard_cuda(m, color, residuals):
    """
    CUDA kernel to update matrix cells based on the chessboard strategy.
    This version accumulates squared differences in a global array for residual calculation.
    The kernel now directly updates the input matrix 'm' and uses an additional matrix to avoid read-write conflicts.
    """
    row, col = cuda.grid(2)
    if row >= 1 and row < m.shape[0] - 1 and col >= 1 and col < m.shape[1] - 1:
        if (row + col) % 2 == color:
            north = m[row-1, col]
            south = m[row+1, col]
            west = m[row, col-1]
            east = m[row, col+1]
            current = m[row, col]
            new_value = (north + south + west + east) / 4
            diff = current - new_value

            # Use atomic addition to safely update the residual sum from multiple threads
            cuda.atomic.add(residuals, 0, diff**2)

            # Directly update the matrix cell
            m[row, col] = new_value

def gauss_seidel_chessboard_cuda(m, maxiter, tol):
    """
    Gauss-Seidel solver using the chessboard strategy and CUDA, returns residual and iteration count.
    """
    m_device = cuda.to_device(m)
    threads_per_block = (16, 16)
    blocks_per_grid_x = math.ceil(m.shape[0] / threads_per_block[0])
    blocks_per_grid_y = math.ceil(m.shape[1] / threads_per_block[1])
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    for iteration in range(maxiter):
        residuals = np.array([0.0], dtype=np.float32)
        residuals_device = cuda.to_device(residuals)

        # Update red cells
        update_cells_by_color_chessboard_cuda[blocks_per_grid, threads_per_block](m_device, 0, residuals_device)
        # Update black cells
        update_cells_by_color_chessboard_cuda[blocks_per_grid, threads_per_block](m_device, 1, residuals_device)

        # Copy the residual back to host to check for convergence
        residuals = residuals_device.copy_to_host()
        if residuals[0] < tol:
            break

    m_result = m_device.copy_to_host()
    return m_result, residuals[0], iteration

# Example usage
if __name__ == '__main__':
    grid_size = 100
    m = init(heat_sources, grid_size-2)
    maxiter = 25000
    tol = 0.00005
    result, final_residual, iterations = gauss_seidel_chessboard_cuda(m, maxiter, tol)
    print(f"Final residual: {final_residual} after {iterations} iterations")
